In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# datasets
trainset = torchvision.datasets.FashionMNIST('../../data',
    download=True,
    train=True,
    transform=transform)
testset = torchvision.datasets.FashionMNIST('../../data',
    download=True,
    train=False,
    transform=transform)

# dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                        shuffle=True, num_workers=2)


testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')


100.0%

Extracting ../../data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ../../data\FashionMNIST\raw


111.0%

Extracting ../../data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ../../data\FashionMNIST\raw


100.0%

Extracting ../../data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ../../data\FashionMNIST\raw


159.1%

Extracting ../../data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ../../data\FashionMNIST\raw
Processing...
Done!


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [4]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.90)
scheduler = optim.lr_scheduler.StepLR(optimizer, 10,gamma=0.9)

In [5]:
running_loss = 0.0
for epoch in range(3):  # loop over the dataset multiple times

    for i, data in enumerate(trainloader, 0):

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 999:    # every 1000 mini-batches...
            scheduler.step()
            print("[%d,%6d] ava_loss: %.5f"%(epoch+1,i+1,running_loss/1000))
            running_loss = 0.0
print('Finished Training')

[1,  1000] ava_loss: 1.83718
[1,  2000] ava_loss: 0.90176
[1,  3000] ava_loss: 0.71733
[1,  4000] ava_loss: 0.67847
[1,  5000] ava_loss: 0.60404
[1,  6000] ava_loss: 0.58556
[1,  7000] ava_loss: 0.52820
[1,  8000] ava_loss: 0.49912
[1,  9000] ava_loss: 0.50080
[1, 10000] ava_loss: 0.47464
[1, 11000] ava_loss: 0.43549
[1, 12000] ava_loss: 0.47869
[1, 13000] ava_loss: 0.43995
[1, 14000] ava_loss: 0.41471
[1, 15000] ava_loss: 0.43339
[2,  1000] ava_loss: 0.40729
[2,  2000] ava_loss: 0.39121
[2,  3000] ava_loss: 0.39643
[2,  4000] ava_loss: 0.39025
[2,  5000] ava_loss: 0.37943
[2,  6000] ava_loss: 0.38009
[2,  7000] ava_loss: 0.39515
[2,  8000] ava_loss: 0.37670
[2,  9000] ava_loss: 0.36269
[2, 10000] ava_loss: 0.36963
[2, 11000] ava_loss: 0.37453
[2, 12000] ava_loss: 0.39974
[2, 13000] ava_loss: 0.36863
[2, 14000] ava_loss: 0.33468
[2, 15000] ava_loss: 0.36482
[3,  1000] ava_loss: 0.34075
[3,  2000] ava_loss: 0.33766
[3,  3000] ava_loss: 0.33489
[3,  4000] ava_loss: 0.34232
[3,  5000] ava

In [6]:
PATH="../../pth/fasion_mnist_test.pth"
torch.save(net.state_dict(),PATH)

FileNotFoundError: [Errno 2] No such file or directory: '../../pth/fasion_mnist_test.pth'

In [ ]:
net = Net()
net.load_state_dict(torch.load(PATH))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network: %d %%' % (
    100 * correct / total))

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))